In [149]:
%matplotlib inline

import matplotlib
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
import matplotlib.pyplot as plt

# Handling Imbalanced Data

## Introduction

In machine learning you typically want to build a model that does what it is suppose to do.

If you have an image classifier, you want your model to correctly classify images.

!["https://www.google.ca/url?sa=i&rct=j&q=&esrc=s&source=images&cd=&cad=rja&uact=8&ved=2ahUKEwjToc6I8pDdAhVNCDQIHayCA9wQjRx6BAgBEAU&url=https%3A%2F%2Fblog.acolyer.org%2F2016%2F04%2F20%2Fimagenet-classification-with-deep-convolutional-neural-networks%2F&psig=AOvVaw3_8QOTYYyy1ixeoaof-d_z&ust=1535584557428370"](../data/img/imagenet_example.png)

So how do you determine how well your model is at doing its job?

With classification, for example, people generally take the proportion of correctly classified images as the model's accuracy score.

$$Accuracy = \frac{\#\ of\ correctly\ classfied\ images}{\#\ of\ total\ classified\ images}$$

A high accuracy score usually equates to a good model.

!["https://plot.ly/~botevmg/5/imagenet-large-scale-visual-recognition-challenge-accuracy.png"](../data/img/imagenet_scores.png)

**BUT... this is not always true...**

## Welcome to healthcare

Let's say you develop a binary classification model that aims to determine a person's HIV status.

If this model is tested on every person in Canada and yields an overall accuracy score of 99.7%, would you consider the model worthy of replacing traditional ways of testing for HIV?

### **NOPE**

The prevelence of HIV/AIDS in Canada is 212 people per 100,000 (0.212%) [1](https://en.wikipedia.org/wiki/HIV/AIDS_in_Canada). This means that 99.788% of Canada's population is HIV negative.

If your model **predicts every person in Canada to be HIV negative**, it would automatically result in an **accuracy score of 99.788%**.

$$Accuracy = \frac{\#\ of\ correctly\ classfied\ images}{\#\ of\ total\ classified\ images}$$

$$Accuracy = \frac{100,000-212}{100,000}$$

$$Accuracy = 0.99788=99.788\%$$

We need to be careful when choosing how we train and validate our machine learning models. We need to make sure that our model does what we expect it to do.

## The Imbalanced Data Conundrum

Let's take another look at our hypothetical model results. This time in the form of a confusion matrix.

![](../data/img/unbalanced_confusion.png)

Even though our accuracy score ($99.8\%$) seems really good, incorrectly telling 212 people they are HIV-, is concerning.

**Our model needs to predict equally well for HIV negative and HIV positive cases. Even if the probability of being HIV+ in Canada is very low.**

## All predictions are equal, but some are more equal than others

The only way to tell the model that it's equally important to correctly classify the minority class (HIV+ cases) as the majority class (HIV- cases), is to **increase the importance of correctly classifying the minority group**.

There are multiple ways of doing this. We will be looking at the following two methods:

* **Resampling data**
    - Undersampling
    - Oversampling
* **Change class weights**

## Case Study: Thoraric Surgery Survival Rate

### Introduction

To make our imbalanced data problem less hypothetical, let's consider the following dataset, which can be found [here](https://archive.ics.uci.edu/ml/datasets/Thoracic+Surgery+Data).

*The data was collected retrospectively at Wroclaw Thoracic Surgery Centre for **patients who underwent major lung resections for primary lung cancer in the years 2007 & 2011**. The Centre is associated with the Department of Thoracic Surgery of the Medical University of Wroclaw and Lower-Silesian Centre for Pulmonary Diseases, Poland, while the research database constitutes a part of the National Lung Cancer Registry, administered by the Institute of Tuberculosis and Pulmonary Diseases in Warsaw, Poland.*

**Objective:** Predict whether a person who undergoes surgery, will be alive or dead 1 year after surgery.

**Patient Properties:**

* DGN: Diagnosis - specific combination of ICD-10 codes for primary and secondary as well multiple tumours if any (DGN3,DGN2,DGN4,DGN6,DGN5,DGN8,DGN1) 
* PRE4: Forced vital capacity - FVC (numeric) 
* PRE5: Volume that has been exhaled at the end of the first second of forced expiration - FEV1 (numeric) 
* PRE6: Performance status - Zubrod scale (PRZ2,PRZ1,PRZ0) 
* PRE7: Pain before surgery (T,F) 
* PRE8: Haemoptysis before surgery (T,F) 
* PRE9: Dyspnoea before surgery (T,F) 
* PRE10: Cough before surgery (T,F) 
* PRE11: Weakness before surgery (T,F) 
* PRE14: T in clinical TNM - size of the original tumour, from OC11 (smallest) to OC14 (largest) (OC11,OC14,OC12,OC13) 
* PRE17: Type 2 DM - diabetes mellitus (T,F) 
* PRE19: MI up to 6 months (T,F) 
* PRE25: PAD - peripheral arterial diseases (T,F) 
* PRE30: Smoking (T,F) 
* PRE32: Asthma (T,F) 
* AGE: Age at surgery (numeric) 
* Risk1Y: 1 year survival period - (T)rue value if died (T,F) 

## Import Data

In [200]:
data = pd.read_csv('../data/thoraric_surgery.csv')
display(data.head())

survival_counts = data.Risk1Yr.value_counts()
print("Number of patients survived: {}\nNumber of patients died: {}".format(*survival_counts))

,DGN,PRE4,PRE5,PRE6,PRE7,PRE8,PRE9,PRE10,PRE11,PRE14,PRE17,PRE19,PRE25,PRE30,PRE32,AGE,Risk1Yr
0,DGN2,2.88,2.16,PRZ1,F,F,F,T,T,OC14,F,F,F,T,F,60,F
1,DGN3,3.40,1.88,PRZ0,F,F,F,F,F,OC12,F,F,F,T,F,51,F
2,DGN3,2.76,2.08,PRZ1,F,F,F,T,F,OC11,F,F,F,T,F,59,F
3,DGN3,3.68,3.04,PRZ0,F,F,F,F,F,OC11,F,F,F,F,F,54,F
4,DGN3,2.44,0.96,PRZ2,F,T,F,T,T,OC11,F,F,F,T,F,73,T


Number of patients survived: 400
Number of patients died: 70


## Clean and Transform Data

To explore the imbalanced data handling methods, we will looking at a vanilla logistic regression model.

Our response variable is patient survival outcome (Is a patient dead or alive after surgery).

This means we are working with a binary classifier.

There are two things we need to do before we can train our model:

* Replace strings with numbers
* Change multiclass variables to binary variables

### Replace strings with numbers

We need to replace all the `"T"`'s and `"F"`'s with `1`'s and `0`'s.

We can't use strings in a numeric equation...

In [201]:
data.replace('F', 0, inplace=True)
data.replace('T', 1, inplace=True)
data.head()

,DGN,PRE4,PRE5,PRE6,PRE7,PRE8,PRE9,PRE10,PRE11,PRE14,PRE17,PRE19,PRE25,PRE30,PRE32,AGE,Risk1Yr
0,DGN2,2.88,2.16,PRZ1,0,0,0,1,1,OC14,0,0,0,1,0,60,0
1,DGN3,3.40,1.88,PRZ0,0,0,0,0,0,OC12,0,0,0,1,0,51,0
2,DGN3,2.76,2.08,PRZ1,0,0,0,1,0,OC11,0,0,0,1,0,59,0
3,DGN3,3.68,3.04,PRZ0,0,0,0,0,0,OC11,0,0,0,0,0,54,0
4,DGN3,2.44,0.96,PRZ2,0,1,0,1,1,OC11,0,0,0,1,0,73,1


### Change multiclass variables to binary variables

For any model with categorical features (opposed to numeric, for example), we need to transform use one-hot encoding to transform multiclass categorical features to binary features.

For example:

*DGN: Diagnosis - specific combination of ICD-10 codes for primary and secondary as well multiple tumours if any (DGN3,DGN2,DGN4,DGN6,DGN5,DGN8,DGN1)*

Instead of saying:

![](../data/img/encoding_before.png)

We need to say:

![](../data/img/encoding_after.png)

Don't worry too muhc about this. Just know that this is what is happening below.

In [202]:
dummy_cols = ['DGN', 'PRE6', 'PRE14']
dummies = pd.get_dummies(data[dummy_cols])

data = data.join(dummies)
data.drop(columns=dummy_cols, inplace=True)
data.head()

,PRE4,PRE5,PRE7,PRE8,PRE9,PRE10,PRE11,PRE17,PRE19,PRE25,...,DGN_DGN5,DGN_DGN6,DGN_DGN8,PRE6_PRZ0,PRE6_PRZ1,PRE6_PRZ2,PRE14_OC11,PRE14_OC12,PRE14_OC13,PRE14_OC14
0,2.88,2.16,0,0,0,1,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,3.40,1.88,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
2,2.76,2.08,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
3,3.68,3.04,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
4,2.44,0.96,0,1,0,1,1,0,0,0,...,0,0,0,0,0,1,1,0,0,0


## Vanilla Logistic Regression with data as-is

As we saw above, the vast majority of patients survived ($400$ out of $470$ patients) the surgery.

If we want the model to maximize the correct number of predictions(in other words, we do not attach more importance to correctly classifying a specific class), we can use train the logistic regression model on the data as-is.

In [233]:
# patient features
X = data.loc[:, data.columns!='Risk1Yr']
# response variable (what we want to predict)
y = data['Risk1Yr']

# split the data into training and testing (70% of the data used for training, and 30% used for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

# train the model
lr = LogisticRegression()
lr.fit(X=X_train, y = y_train)

# test the model on unseen test data
predictions = lr.predict(X_test)

conf_mat = confusion_matrix(y_true=y_test, y_pred=predictions)
test_accuracy = (conf_mat[0, 0] + conf_mat[1, 1]) / np.sum(conf_mat)

print('Accuracy: {0:.2f}%'.format(test_accuracy*100))

Accuracy: 81.56%


So in terms of *number of correct predictions*, our model seems to be doing quite well.

Let's take a look at the confusion matrix, which will reveal a bit more about how our model does its classifications.

In [234]:
pd.DataFrame(conf_mat, index=['Actual_Survived', 'Actual_Died'], columns=['Predicted_Survived', 'Predicted_Died'])

,Predicted_Survived,Predicted_Died
Actual_Survived,114,2
Actual_Died,24,1


The confusion matrix is showing that our model rarely ever predict that a patient will die. Since it's a safer bet to predict that a patient has survived (due to the imbalance), the model choses to do this when there is a certain level of uncertainty.

## Method 1: Resampling Data

### Undersampling

Undersampling is the process of **reducing the number of samples of the majority class** during model training.

We aim to end up with a reduced number of majority class samples equal to the number of samples in the minority class.

This will result in balanced data and cause the model to pay more attention to the minority class (since it now has just as many samples as the majority class).

In [240]:
# determine the number of samples in the minority class
label_min_count = y.value_counts().min()
# determine the label of the minority class
label_min = y.value_counts().idxmin()

# create data subsets for the 2 classes
data_max_label = data[data.Risk1Yr != label_min]
data_min_label = data[data.Risk1Yr == label_min]

# reduce the number of samples in the majority class subset
data_max_downsample = data_max_label.sample(n = label_min_count, replace=False, random_state=1234)

# merge the two subsets to give us the balanced dataset
data_balanced = data_min_label.append(data_max_downsample)

survival_counts = data.Risk1Yr.value_counts()
print("Original Data:\nNumber of patients survived: {}\nNumber of patients died: {}\n".format(*survival_counts))

survival_counts = data_balanced.Risk1Yr.value_counts()
print("Undersampled, Balanced Data:\nNumber of patients survived: {}\nNumber of patients died: {}".format(*survival_counts))

Original Data:
Number of patients survived: 400
Number of patients died: 70

Undersampled, Balanced Data:
Number of patients survived: 70
Number of patients died: 70


In [236]:
# patient features
X = data_balanced.loc[:, data_balanced.columns!='Risk1Yr']
# response variable (what we want to predict)
y = data_balanced['Risk1Yr']

# split the data into training and testing (70% of the data used for training, and 30% used for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

# train the model
lr = LogisticRegression()
lr.fit(X=X_train, y = y_train)

# test the model on unseen test data
predictions = lr.predict(X_test)

conf_mat = confusion_matrix(y_true=y_test, y_pred=predictions)
test_accuracy = (conf_mat[0, 0] + conf_mat[1, 1]) / np.sum(conf_mat)

print('Accuracy: {0:.2f}%'.format(test_accuracy*100))

Accuracy: 57.14%


In [238]:
pd.DataFrame(conf_mat, index=['Actual_Survived', 'Actual_Died'], columns=['Predicted_Survived', 'Predicted_Died'])

,Predicted_Survived,Predicted_Died
Actual_Survived,11,12
Actual_Died,6,13


We can see that our results are more balanced, but our overall number of correct predictions has decreased quite a bit.

One of the main issues here, is that we lost a lot of training data by undersampling. Since we didn't have many samples to start off with, we end up with a small training dataset.

For this reason, it makes more sense to oversample, rather than undersample.

### Oversampling

Oversampling is the process of **increasing the number of samples of the minority class** during model training.

We aim to end up with a increased number of minority class samples equal to the number of samples in the majority class.

This will result in balanced data and cause the model to pay more attention to the minority class (since it now has just as many samples as the majority class), while still using all the data we have available.

In [242]:
# determine the  number of samples in the majority class
y = data['Risk1Yr']
label_max_count = y.value_counts().max()
# determine the label of the majority class
label_max = y.value_counts().idxmax()

# increase the number of samples in the minority class subset
data_min_upsample = data_min_label.sample(n = label_max_count, replace=True, random_state=1234)

# merge the two subsets to give us the balanced dataset
data_balanced = data_max_label.append(data_min_upsample)

survival_counts = data.Risk1Yr.value_counts()
print("Original Data:\nNumber of patients survived: {}\nNumber of patients died: {}\n".format(*survival_counts))

survival_counts = data_balanced.Risk1Yr.value_counts()
print("Oversampled, Balanced Data:\nNumber of patients survived: {}\nNumber of patients died: {}".format(*survival_counts))

Original Data:
Number of patients survived: 400
Number of patients died: 70

Oversampled, Balanced Data:
Number of patients survived: 400
Number of patients died: 400


In [243]:
# patient features
X = data_balanced.loc[:, data_balanced.columns!='Risk1Yr']
# response variable (what we want to predict)
y = data_balanced['Risk1Yr']

# split the data into training and testing (70% of the data used for trainning, and 30% used for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

# train the model
lr = LogisticRegression()
lr.fit(X=X_train, y = y_train)

# test the model on unseen test data
predictions = lr.predict(X_test)

conf_mat = confusion_matrix(y_true=y_test, y_pred=predictions)
test_accuracy = (conf_mat[0, 0] + conf_mat[1, 1]) / np.sum(conf_mat)

print('Accuracy: {0:.2f}%'.format(test_accuracy*100))

Accuracy: 67.50%


In [244]:
pd.DataFrame(conf_mat, index=['Actual_Survived', 'Actual_Died'], columns=['Predicted_Survived', 'Predicted_Died'])

,Predicted_Survived,Predicted_Died
Actual_Survived,85,47
Actual_Died,31,77


## Extraaaa

There are a number of ways to tell our model that it's equally important to 

**If HIV prevelance was higher, would it have been more obvious that our model is not as good as we thought?**

Let's say we use the HIV prevelance data of pregnant women in KZN province, South Africa to validate our model.

[HIV prevelance of pregnant women in KZN, South Africa](https://en.wikipedia.org/wiki/HIV/AIDS_in_South_Africa) is $37\%$. And let's say we again test $100,000$ people.

![](../data/img/kzn_confusion.png)


Now our model is incorrectly telling 37,000 people they are HIV- ($Accuracy\ = 67\%$).

In this case it is more evident that our model is actually doing a pretty poor job, even though it is the same model.

**This serves as an indication that having a more uniform distribution (balanced data) shows a more realistic picture of our model.**

In [220]:
# score = precision_recall_fscore_support(y_true=y_test, y_pred=predictions, average='binary')
# score